In [ ]:
from mtcnn import MTCNN
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
# Run this cell if using all of the data
# 'Face Images' contains both the cleaned FER2013 dataset and the cleaned CMU Face Images dataset
df = pd.read_csv("Face Images.csv")

In [9]:
# Run this cell if using only a subset of the data

frac = pd.read_csv("Face Images.csv").sample(frac=0.01)
whole = pd.read_csv("Face Images.csv")

disgust = whole[whole.emotion == 1]
frac = frac[frac.emotion != 1]
df = pd.concat([frac, disgust]) # df is 25% of the whole dataset + the disgust samples not taken in the 25% (since there are so few, use all of them)

In [11]:
def crop_image(image, box):
    # Input: image: np array of pixels (image) to crop
    #        box: a dictionary returned by detect_faces() containing coordinates/width/height of the box outlining the detected face
    # Output: an np array of pixels representing the cropped image
    x, y, width, height = box[0], box[1], box[2], box[3] # x,y,w,h of the box containing a detected face
    return image[y:y+height, x:x+width]

In [12]:
def resize_image(image):
    # Input: image: an np array of pixels
    # Output: an np array representing the input image resized to 224x224 px
    size = (224, 224)
    return np.array(Image.fromarray(np.squeeze(image, axis=2)).resize(size))

In [ ]:
# Modified: 6.54 for MTCNN and saving the images
#Original:

# For each image in the df, use the mtcnn to detect the face, then crop the image to just the face portion, and then resize to a constant size of 224x224 px
images = []
count = 0
detector = MTCNN()

for index, row in df.iterrows():
    original_image = np.fromstring(row["pixels"], dtype=int, sep=" ").reshape(row["height"], row["width"], 1).astype("float32")
    img = np.tile(original_image, [1,1,3]) # this MTCNN library only works with colour images (imgs with 3 channels), but grayscale images have only 1 channel. This pads the img to make the image have 3 channels
    output = detector.detect_faces(img)
    if len(output) == 1: # 1 face detected
        resized_image = resize_image(crop_image(original_image, output[0]["box"]))
        images.append([row["emotion"], resized_image])

In [ ]:
# Display a cropped image
plt.imshow(np.array(images[7151][1]).astype('float32'))

In [16]:
# Save the images to the directory corresponding to their labels
expressions = {0:"Angry", 1:"Disgust", 2:"Fear", 3:"Happy", 4:"Sad", 5:"Surprise", 6:"Neutral"}

count = 0

for image in images:
    label = image[0]
    pixels = image[1]

    # convert pixels to Image class
    im = Image.fromarray(pixels).convert('L')

    # same image to the test directory
    im.save(f"Abcd Efghij/{expressions[label]}/img_{count}.jpg", "JPEG")
    count += 1